## RED LSTM (LONG SHORT-TERM MEMORY - Memoria Larga a Corto Plazo) 

Este código crea un modelo LSTM que aprende a predecir el siguiente carácter en un texto dado basado en los cinco caracteres anteriores. Después de entrenar el modelo, genera texto automáticamente comenzando desde una secuencia inicial aleatoria.

Este ejercicio permite demostrar cómo un LSTM puede aprender dependencias en datos secuenciales como el texto. Se usa un texto corto como ejemplo y se prepara el código para predecir el siguiente carácter dado una secuencia de caracteres anteriores.

Se sigue una pauta similar a la llevada a cabo en el notebook 07_RNA_RedRNN.ipynb

In [1]:
import warnings
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense


2025-03-19 22:52:05.914033: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-19 22:52:05.958052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742424726.007017   62089 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742424726.020697   62089 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 22:52:06.073006: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Desactivamos warnings

In [2]:
warnings.filterwarnings("ignore")

### Definición del conjunto de datos

Para que un modelo de aprendizaje automático pueda procesar texto, primero debemos convertir los caracteres en una forma que el modelo pueda entender, es decir, números.   
En este caso, tomamos la frase y generamos directamente el conjunto de caracteres que la forman y la convertimos a un conjunto de números enteros.

In [2]:
# Texto de ejemplo
# text = "fractured world tested the hope of a young president"
text = "en un lugar de la mancha de cuyo nombre no quiero acordarme"
# Convertir caracteres a enteros
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

### Preparación de datos para ser usados por la red LSTM

Un modelo LSTM necesita conocer no solo el estado actual (por ejemplo, un carácter en el texto), sino también algunos estados anteriores para hacer una predicción efectiva. Esto es lo que le permite aprender y entender el contexto o la secuencia en los datos.

En este caso, se define una longitud de secuencia de 5 (seq_length). De esta forma, se va a utilizar secuencias de 5 caracteres para predecir el siguiente carácter.   
   
Creamos todas las posibles subsecuencias de 5 caracteres del texto y el carácter siguiente a cada subsecuencia será el **carácter objetivo** que el modelo intentará predecir.

In [3]:
# Preparar los datos para el LSTM
seq_length = 5
dataX = []
dataY = []
for i in range(0, len(text) - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

### Reformateo de los Datos de Entrada para el LSTM
Los modelos LSTM en TensorFlow esperan una entrada con la forma [muestras, pasos de tiempo, características]:

- **Muestras**: Número total de secuencias de ejemplo.
- **Pasos de tiempo**: Número de unidades de tiempo en la secuencia; en este caso, es la longitud de la secuencia de caracteres (5).
- **Características**: Número de características en cada paso de tiempo; en este ejemplo, es 1 porque tenemos un carácter por paso de tiempo.

Además, se normalizan los valores de entrada dividiendo por el número total de caracteres únicos para ayudar al modelo a converger más rápido durante el entrenamiento.

In [4]:
n_patterns = len(dataX)
X = np.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(len(chars))

### Codificación One-Hot para la Salida

La salida se codifica como un vector one-hot, lo que significa que cada carácter posible se representa como un vector binario con un solo bit activo (1) y todos los demás inactivos (0). Esto es necesario para la clasificación, donde cada entrada puede ser clasificada como uno de los caracteres posibles.

In [5]:
y = tf.keras.utils.to_categorical(dataY)

### Definición de la arquitectura del modelo de red LSTM

Definición y compilación del modelo de red LSTM

In [6]:
# Definir el modelo LSTM
model = Sequential([
    LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    LSTM(128),
    Dense(y.shape[1], activation='softmax')
])

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam')

2025-03-19 22:53:09.453250: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Entrenamiento

In [7]:
# Entrenar el modelo
model.fit(X, y, epochs=100, batch_size=1, verbose=2)


Epoch 1/100


54/54 - 5s - 95ms/step - loss: 2.8368
Epoch 2/100
54/54 - 1s - 14ms/step - loss: 2.7292
Epoch 3/100
54/54 - 1s - 13ms/step - loss: 2.6668
Epoch 4/100
54/54 - 1s - 14ms/step - loss: 2.6630
Epoch 5/100
54/54 - 1s - 14ms/step - loss: 2.6381
Epoch 6/100
54/54 - 1s - 14ms/step - loss: 2.6517
Epoch 7/100
54/54 - 1s - 14ms/step - loss: 2.6301
Epoch 8/100
54/54 - 1s - 14ms/step - loss: 2.6164
Epoch 9/100
54/54 - 1s - 14ms/step - loss: 2.6264
Epoch 10/100
54/54 - 1s - 14ms/step - loss: 2.6300
Epoch 11/100
54/54 - 1s - 14ms/step - loss: 2.6402
Epoch 12/100
54/54 - 1s - 14ms/step - loss: 2.5949
Epoch 13/100
54/54 - 1s - 15ms/step - loss: 2.5992
Epoch 14/100
54/54 - 1s - 15ms/step - loss: 2.5532
Epoch 15/100
54/54 - 1s - 14ms/step - loss: 2.5237
Epoch 16/100
54/54 - 1s - 14ms/step - loss: 2.4653
Epoch 17/100
54/54 - 1s - 14ms/step - loss: 2.4586
Epoch 18/100
54/54 - 1s - 13ms/step - loss: 2.4234
Epoch 19/100
54/54 - 1s - 14ms/step - loss: 2.3856
Epoch 20/100
54/54 - 1s - 14ms/step - loss: 2.3610
E

### Pruebas y predicción

Primero, el código selecciona una secuencia inicial aleatoria del conjunto de datos de entrenamiento. Esta secuencia servirá como "semilla" para iniciar el proceso de generación de texto.    
La idea es proporcionar al modelo un contexto inicial basado en el cual él puede comenzar a generar el siguiente carácter.   

- **start** es un índice aleatorio que determina el punto de partida en el conjunto de datos.
- **pattern** es la secuencia de caracteres (en forma de índices numéricos) extraída de *dataX* usando el índice *start*.

A continuación, se imprime esta secuencia convertida de nuevo a caracteres para visualizar la semilla con la que se inicia la generación.

In [10]:
# Demostrar la generación de texto
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Semilla:")
print("\"", ''.join([chars[value] for value in pattern]), "\"")

Semilla:
" un lu "


#### Generación de caracteres

El código arranca un bucle donde, en cada iteración, se intenta predecir el siguiente carácter basado en la secuencia actual (**pattern**).   

Seguidamente se actualiza la secuencia moviéndola un paso hacia adelante, incluyendo el nuevo carácter predicho al final y descartando el primer carácter para mantener la longitud constante.   

- **x** reformatea la secuencia *pattern* para que tenga el formato adecuado para el modelo: [1, longitud de la secuencia, 1], y la normaliza dividiendo por el número de caracteres únicos.
- **prediction** es el vector de salida del modelo, donde cada elemento representa la probabilidad de que un carácter sea el siguiente en la secuencia.
- **index** es el índice del carácter con la mayor probabilidad en prediction.
- **result** es el carácter correspondiente al índice index, que se añade a la secuencia generada y se imprime.
- La secuencia **pattern** se actualiza añadiendo el índice del carácter predicho al final y eliminando el primer elemento para mantener el tamaño.   

Este bucle permite visualizar cómo el modelo, basándose en una semilla inicial y su "memoria" de lo aprendido durante el entrenamiento, puede generar texto que sigue alguna forma de estructura gramatical y léxica, dependiendo de la complejidad del texto original y la duración del entrenamiento.

In [12]:
# Generar caracteres
for i in range(50):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(chars))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = chars[index]
    seq_in = [chars[value] for value in pattern]
    print(result, end='')
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nTerminado.")

dduadeo he da maychnu rco r  derdduadeo he da mayc
Terminado.
